### 神经元模型

![image.png](attachment:image.png)

### 感知机与多层网络
>感知机由两层神经元组成。输入层接收外界输入信号后传递给输出层，输出层是 M-P 神经元，亦称位"阈值逻辑单元(threshold logic uint)"

感知机能够轻易的实现逻辑与、或、非运算。

$exmple$:

In [25]:
import numpy as np

In [29]:
# 阶跃函数
def unit_step(x):
    y = x>=0
    return y.astype(np.int)

# 逻辑运算
def perc_logic(x,w,theta):
    y = x@w.T-theta
    out = unit_step(y)
    return out
x = np.array([[1,1],[0,1],[0,0],[1,0]])
# 逻辑与
w = np.array([1,1])
theta = 2
y = perc_logic(x,w,theta)

In [ ]:
print(x.T)
print(y)

[[1 0 0 1]
 [1 1 0 0]]
[1 0 0 0]


这里的权重是我们自己给的。一般的，给定训练数据集，权重 $w_i$ 以及阈值 $\theta$ 可通过学习得到。感知机权重会根据如下方式进行调整：
$$
w_i\gets w_i+\Delta w_i\\
\Delta w_i=\eta (y-\hat y)x_i
$$
其中，$\eta \in (0,1)$ 称为学习率(learning rate)。感知机只能处理一些线性可分问题，其学习能力十分有限。

In [229]:
# 感知机实现 逻辑与的学习
import numpy as np
# 阶跃函数
def unit_step(x):
    y = x>=0
    return y.astype(np.int)

# 创造数据集
def setdata(n):
    x1 = np.random.randint(0,2,n)
    x2 = np.random.randint(0,2,n)
    y = np.logical_and(x1,x2)
    data = [x1,x2,y]
    return np.array(data).T

# 感知机
class prec:

    def __init__(self,w,lr,expoch):
        self.w = w
        self.lr = lr
        self.expoch = expoch

    def forward(self,x):
        y1 = x@self.w.T
        h1 = unit_step(y1)
        out = h1
        return out
    
    # 梯度更新
    def gradient_fit(self,x,y):
        out = self.forward(x)
        gradient_w = np.sum(self.lr*(y-out)*x,axis=0)
        self.w = self.w+gradient_w.T

    def run(self,x,y):
        for step in range(self.expoch):
            self.gradient_fit(x,y)


n = 1200
rate = 0.8
offset = int(n*rate)

data = setdata(n)
b = np.ones((n,1))
data =  np.hstack((data,b))

x_train,y_train,x_test,y_test = data[:offset,[0,1,-1]],data[:offset,2],data[offset:,[0,1,-1]],data[offset:,2]
y_train = y_train.reshape((offset,1))
y_test = y_test.reshape((n-offset,1))

init_w = np.zeros((1,3))
lr = 0.001
expoch = 1000

net = prec(init_w,lr,expoch)
net.run(x_train,y_train)

pre = unit_step(x_test@net.w.T)
acc = 1-np.sum(np.abs(y_test-pre))/len(y_test)

print(net.w)
print(acc)

[[ 0.264  0.272 -0.442]]
1.0


### 误差逆传播(BP)算法

![image.png](attachment:image.png)

假设隐含层和输出层神经元都使用 $Sigmoid$ 函数，则对训练例 $(x_k,y_k),y_k=(y_1^k,y_2^k,...,y_l^k)$ 有：

$$
\hat y_j^k = f(\beta_j - \theta_j)
$$

在网络上 $(x_k,y_k)$ 处的均方误差为：
$$
E_k = \frac{1}{2}\sum_{j=1}^{l}(\hat y_j^k - y_j^k)^2
$$

上面网络中需要学习的参数有：$d\times q + q\times l + q+l$ 个。$BP$ 是一个迭代学习算法，在迭代的每一轮中采用广义的感知机学习规则对参数进行更新估计，任意参数 $v$ 的更新估计式为：
$$
v \gets v + \Delta v
$$

$BP$ 算法在基于梯度下降(gradient descent) 策略更新时，以目标的负梯度方向对参数进行调整。例如，对于上述的 $E_k$ ,给定学习率 $\eta$ 时，有：
$$
\Delta w_{hj}=-\eta \frac{\partial E_k}{\partial w_{hj}}\\
\frac{\partial E_k}{\partial w_{hj}}=\frac{\partial E_k}{\partial \hat y_{j}^k}
·\frac{\partial \hat y_{j}^k}{\partial \beta_j}·\frac{\partial \beta_j}{\partial w_{hj}}\\
\frac{\partial \beta_j}{\partial w_{hj}}=b_h
$$

该网络中的 $Sigmoid$ 函数具如下良好的性质：
$$
f'(x)=f(x)·f(1-f(x))
$$

于是：
$$
g_i = -\frac{\partial E_k}{\partial \hat y_{j}^k}
·\frac{\partial \hat y_{j}^k}{\partial \beta_j}\\
=-(\hat y_j^k-y_j^k)f'(\beta_j-\theta_j)\\
=\hat y_j^k(1-\hat y_j^k)(y_j^k-\hat y_j^k)
$$

最后得到 $BP$ 算法中关于 $w_{hj}$ 的更新公式：
$$
\Delta w_{hj}=\eta g_ib_h
$$

类似的：
$$
\Delta \theta_j = -\eta g_i\\
\Delta v_{ih} = \eta e_hx_i\\
\Delta \gamma_h=-\eta e_h\\
$$
其中：
$$
e_h = -\frac{\partial E_k}{\partial b_h}·\frac{\partial b_h}{\partial \alpha_h}\\
=b_h(1-b_h)\sum_{j=1}^{l}w_{hj}g_i
$$

![image.png](attachment:image.png)

需要注意的是，$BP$ 算法的目标是要最小化训练集 $D$ 上的累计误差：
$$
E = \frac{1}{m}\sum_{k=1}^{m}E_k
$$

缓解过拟合策略：

- 早停
>将数据集分成训练集和验证集，训练集用来计算梯度，更新连接权和阈值，验证集用来估计误差，若训练集误差降低但验证集误差升高，则停止训练，同时返回具有最小验证集误差的连接权和阈值。

- 正则化
>在误差目标函数中增加一个用于描述网络复杂度的部分，例如：$E=\lambda \frac{1}{m}\sum_{k=1}^{m}E_k+(1-\lambda)\sum_{i}w_i^2$，$\lambda \in (0,1)$ 用于对经验误差与网络复杂度这两项进行折中，常通过交叉验证法来估计。

### 避免局部最小

- 以多组不同参数值初始化多个神经网络
- 使用“模拟退火”
- 使用随机梯度下降
- 遗传算法

### 其他常见神经网络

- $RBF$ 网络

- $ART$ 网络

- $SOM$ 网络

- 级联相关网络

- $Elman$ 网络

- $Boltzmann$ 机

### 一个单隐层 $BP$ 算法的实现

In [174]:
import numpy as np
import matplotlib.pyplot as plt

In [175]:
plt.style.use('ggplot')
%matplotlib inline
# 对中文的支持
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [264]:
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y

class SimpleNet:

    def __init__(self,w,b,lr,expoch):

        self.w = w
        self.b = b
        self.lr = lr
        self.expoch = expoch

    def forwad(self,x):

        y1 = self.w[0].T@x+self.b[0]
        h1 = sigmoid(y1)
        y2 = self.w[1].T@h1+self.b[1]
        h2 = sigmoid(y2)
        out = h2
        return out,h1

    def loss(self,y,ylabel):

        Ek = np.sum((y-ylabel)**2)/2

        return Ek

    def gradient_gj_eh(self,x,y,ylabel,h1):

        gj = y*(1-y)*(ylabel-y)

        eh = h1*(1-h1)*(self.w[1])@gj
        grad_w1 = self.lr*x@eh.T
        grad_w2 = self.lr*h1@gj.T

        grad_b1 = -self.lr*eh
        grad_b2 = -self.lr*gj

        self.w[0] += grad_w1
        self.w[1] += grad_w2

        self.b[0] += grad_b1
        self.b[1] += grad_b2


    def fit(self,x_train,y_train):
        m = x_train.shape[-1]

        for step in range(self.expoch):
            ls = 0
            for k in range(m):
                x = np.array(x_train[:,k],ndmin=2).T
                ylabel = np.array(y_train[:, k], ndmin=2).T
                out,h1 = self.forwad(x)
                ls += self.loss(out,ylabel)

                self.gradient_gj_eh(x,out,ylabel,h1)

            E = ls / m
            if step%50==0:
                print(E,out)

In [265]:
xm = 10 # 输入层样本数量
xd = 2 # 输入层样本属性（输入层神经元数量）
hq = 5 # 隐含层神经元数量
outl = 1 # 输出层神经元数量

In [266]:
x = np.ones((xd,xm))

y = np.ones((outl,xm))

In [267]:
w1 = np.zeros((xd,hq))
w2 = np.zeros((hq,outl))
w = [w1,w2]

b1 = np.zeros((hq,1))
b2 = np.zeros((outl,1))
b = [b1,b2]

In [268]:
net = SimpleNet(w,b,lr=0.01,expoch=100)

In [269]:
net.fit(x,y)

0.1248243769260496 [[0.5007027]]
0.10554248349200798 [[0.54095249]]
